In [41]:
import warnings
warnings.filterwarnings('ignore')


# 교차 검증과 그리드 서치
- 머신러닝을 사용할때 모델의 정확도를 측정하기 위해 반드시 사용해야 하는 방법
- 딥러닝시에는 데이터의 크기가 크므로 이 방법은 사용할 필요가 없다.

In [42]:
import pandas as pd
wine = pd.read_csv("../Data/wine.csv")
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [43]:
# Feature, Target
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

# 검증 세트 추가

In [44]:
# 전체 셋 중에 훈련과 테스트 셋을 8:2의 기준으로 분리한다.
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [45]:
# 훈련세트 중 훈련세트
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

In [47]:
# 훈련세트, 검증세트, 테스트 세트의 크기 구하기
print('훈련셋 :', sub_input.shape, 
'검증셋 :', val_input.shape, 
'테스트셋 :',sub_input.shape)

훈련셋 : (4157, 3) 검증셋 : (1040, 3) 테스트셋 : (4157, 3)


In [48]:
# 훈련세트와 검증세트를 결정 트리로 모델 만들기
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print("Train score :", dt.score(sub_input, sub_target))
print("Valid score :", dt.score(val_input, val_target))

Train score : 0.9971133028626413
Valid score : 0.864423076923077


---
## 교차검증

In [49]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.00661612, 0.00610304, 0.00644708, 0.0070951 , 0.00599504]),
 'score_time': array([0.000808  , 0.00066996, 0.00091195, 0.00061488, 0.00060391]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [50]:
import numpy as np
np.mean(scores['test_score'])

0.855300214703487

---
## KFold : 분할기를 사용한 교차검증

In [51]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=5) # 5 = 기본 n_splits값
scores = cross_validate(dt, train_input, train_target, cv=splitter)
scores

{'fit_time': array([0.01206589, 0.00679088, 0.00612092, 0.00669789, 0.00596499]),
 'score_time': array([0.00074005, 0.00073099, 0.00071716, 0.00072503, 0.00062704]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [52]:
np.mean(scores['test_score'])

0.855300214703487

In [53]:
# KFold의 Fold를 10개로 나누어서 교차검증
    # shuffle true로 하면 좋은게 과적합을 예방할 수 있다.
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # 5 = 기본 n_splits값
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8574181117533719